In [ ]:

# FinCast Forecast Demo
# This notebook validates config, runs the FinCast trainer on sample data, and visualizes yield/ft².

import pandas as pd
from pathlib import Path
from sportai.fincast.validator import load_json, validate_config
from sportai.fincast.trainer import FinCastTrainer

BASE = Path.cwd().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd()
cfg_path = BASE / 'sample_fincast_config.json'
schema_path = BASE / 'fincast_config.schema.json'

schema = load_json(schema_path)
cfg = load_json(cfg_path)
errs = validate_config(cfg, schema)
assert not errs, f'Config invalid: {errs}'

assets = pd.read_csv(BASE / 'sample_data' / 'assets.csv')
history = pd.read_csv(BASE / 'sample_data' / 'history.csv')

trainer = FinCastTrainer(cfg)
res = trainer.forecast(history, horizon_days=14, assets=assets)
fc = res.forecasts

display(fc.head())

# Plot simple trends
import matplotlib.pyplot as plt

plt.figure()
fc.groupby('date')['yield_per_sqft'].mean().plot()
plt.title('Avg Yield per ft² (14d)')
plt.xlabel('Date'); plt.ylabel('Yield/ft²')
plt.show()

plt.figure()
fc.groupby('date')['occupancy_pct'].mean().plot()
plt.title('Avg Occupancy (14d)')
plt.xlabel('Date'); plt.ylabel('Occupancy')
plt.show()

# Save for dashboard
out = BASE / 'forecast_14d.csv'
fc.to_csv(out, index=False)
print('Saved forecast to', out)
